In [1]:
# https://arxiv.org/abs/cond-mat/0703314
# https://arxiv.org/abs/0808.3779

# https://github.com/scipy/scipy/issues/7151
# https://apps.dtic.mil/sti/pdfs/AD1004183.pdf
# https://www.codeproject.com/Articles/21282/Compute-Permanent-of-a-Matrix-with-Ryser-s-Algorit

# https://rosettacode.org/wiki/Determinant_and_permanent
# https://codegolf.stackexchange.com/questions/97060/calculate-the-permanent-as-quickly-as-possible

# https://stackoverflow.com/questions/38738835/generating-gray-codes
# https://qiita.com/b1ueskydragon/items/75cfee42541ea723080c

# https://qiita.com/phdax/items/3064de264c7933bab2f5
# https://web.archive.org/web/20190108235115/https://www.hackersdelight.org/hdcodetxt/pop.c.txt
# http://graphics.stanford.edu/~seander/bithacks.html#CountBitsSetParallel
# https://stackoverflow.com/questions/9829578/fast-way-of-counting-non-zero-bits-in-positive-integer

# https://stackoverflow.com/questions/22227595/convert-integer-to-binary-array-with-suitable-padding

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from numba import jit
import time

In [3]:
# https://github.com/XanaduAI/thewalrus/blob/master/thewalrus/_permanent.py

@jit(nopython=True)
def perm_bbfg(M):  # pragma: no cover
    """
    Returns the permanent of a matrix using the bbfg formula in Gray ordering
    The code is a re-implementation from a Python 2 code found in
    `Permanent code golf
    <https://codegolf.stackexchange.com/questions/97060/calculate-the-permanent-as-quickly-as-possible>`_
    using Numba.
    Args:
        M (array) : a square array.
    Returns:
        float or complex: the permanent of a matrix ``M``
    """

    n = len(M)
    if n == 0:
        return M.dtype.type(1.0)
    row_comb = np.sum(M, 0)
    total = 0
    old_gray = 0
    sign = +1
    binary_power_dict = np.array([2**i for i in range(n)])
    num_loops = 2 ** (n - 1)
    for bin_index in range(1, num_loops + 1):
        reduced = np.prod(row_comb)
        total += sign * reduced
        new_gray = bin_index ^ (bin_index // 2)
        gray_diff = old_gray ^ new_gray
        gray_diff_index = np.searchsorted(binary_power_dict,gray_diff)
        new_vector = M[gray_diff_index]
        direction = 2 * ((old_gray > new_gray) - (old_gray < new_gray))
        for i in range(n):
            row_comb[i] += new_vector[i] * direction
        sign = -sign
        old_gray = new_gray
    return total / num_loops

In [4]:
@jit(nopython=True)
def calc_rsks(L):
    rs = np.arange(1,L+1) ## from 1 to L
    ks = np.linspace(np.pi/(L+1.0),L*np.pi/(L+1.0),L)
    return rs,ks

@jit(nopython=True)
def calc_eks(J,ks):
    return -2.0*J*np.cos(ks)

@jit(nopython=True)
def calc_xkrs(L,ks,rs):
    return np.sqrt(2.0/(L+1.0))*np.sin(np.outer(ks,rs))

@jit(nopython=True)
def calc_ys(t,eks,xkrs):
    xc = (1.0+0.0j)*xkrs # for numba
    expeks = np.diag(np.exp(1j*t*eks))
    return xc @ expeks @ xc

# @jit(nopython=True)
# def calc_fs(ys,system):
#     diag = (1.0+0.0j)*np.diag(system)
#     return ys @ diag @ ys.T.conjugate()

@jit(nopython=True)
def calc_fs_cdw(ys,system,period=1):
    diag = (1.0+0.0j)*np.diag(system)
    ycuts = np.copy(ys[::period,:]) ## avoid NumbaPerformanceWarning to make contiguous arrays by deep copy
#    ycuts = ys[::period,:]
    return ycuts @ diag @ ycuts.T.conjugate()

@jit(nopython=True)
def calc_z(L,system,t,eks,xkrs,period=1):
    ys = calc_ys(t,eks,xkrs)
#    fs = calc_fs(ys,system)
    fs = calc_fs_cdw(ys,system,period=period)
    dlt = np.eye(fs.shape[0],dtype=np.complex128)
    return np.vstack( (np.hstack((fs,dlt-fs)), np.hstack((dlt-fs,fs))) )

@jit(nopython=True)
def calc_z_2(L,system,t,eks,xkrs,period=1):
    ys = calc_ys(t,eks,xkrs)
#    fs = calc_fs(ys,system)
    fs = calc_fs_cdw(ys,system,period=period)
    dlt = np.eye(fs.shape[0],dtype=np.complex128)
    return np.vstack( (np.hstack((fs,dlt-fs)), np.hstack((dlt-fs,fs))) ), fs

@jit(nopython=True)
def get_dat(L,system,J,ts,period=1):
    rs, ks = calc_rsks(L)
    eks = calc_eks(J,ks)
    xkrs = calc_xkrs(L,ks,rs)
    data = np.zeros((len(ts),2),dtype=np.float64)
    for i,t in enumerate(ts):
        matz = calc_z(L,system,t,eks,xkrs,period=period)
        data[i] = np.array([t,-np.log(perm_bbfg(matz).real)])
    return data

In [5]:
## Mott insulator initial
## matrix size: 2L*2L
## len(matz[0,:]) = 2L

period = 1
J = 1.0
#ts = np.linspace(0,20.0,201)
ts = [0.0,10.0]
#Ls = [2,4,6,8,10,12,14,16,18,20]
#Ls = np.arange(2,42,2)
Ls = [2,12,22]

for i,L in enumerate(Ls):
    rs, ks = calc_rsks(L)
    eks = calc_eks(J,ks)
    xkrs = calc_xkrs(L,ks,rs)
    system = np.array([1.0 if i<L//2 else 0.0 for i in range(L)])
    for j,t in enumerate(ts):
        #matz = calc_z(L,system,t,eks,xkrs,period=period)
        matz, fs = calc_z_2(L,system,t,eks,xkrs,period=period)
        #print(np.sum(np.abs(fs.conjugate().T-fs)))
        print(L,t,np.linalg.eigvalsh(matz))
        print("#",2.0*np.linalg.eigvalsh(fs)-1.0)

2 0.0 [-1.  1.  1.  1.]
# [-1.  1.]
2 10.0 [-1.  1.  1.  1.]
# [-1.  1.]
12 0.0 [-1. -1. -1. -1. -1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.]
# [-1. -1. -1. -1. -1. -1.  1.  1.  1.  1.  1.  1.]
12 10.0 [-1. -1. -1. -1. -1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.]
# [-1. -1. -1. -1. -1. -1.  1.  1.  1.  1.  1.  1.]
22 0.0 [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.]
# [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.]
22 10.0 [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.]
# [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.]


In [6]:
## CDW 010101... initial
## matrix size: L*L
## len(matz[0,:]) = L

period = 2
J = 1.0
#ts = np.linspace(0,20.0,201)
ts = [0.0,10.0]
#Ls = [2,4,6,8,10,12,14,16,18,20]
#Ls = np.arange(2,42,2)
Ls = [2,12,22]

for i,L in enumerate(Ls):
    rs, ks = calc_rsks(L)
    eks = calc_eks(J,ks)
    xkrs = calc_xkrs(L,ks,rs)
    system = np.array([1.0 if i<L//2 else 0.0 for i in range(L)])
    for j,t in enumerate(ts):
        #matz = calc_z(L,system,t,eks,xkrs,period=period)
        matz, fs = calc_z_2(L,system,t,eks,xkrs,period=period)
        #print(np.sum(np.abs(fs.conjugate().T-fs)))
        print(L,t,np.linalg.eigvalsh(matz))
        print("#",2.0*np.linalg.eigvalsh(fs)-1.0)

2 0.0 [1. 1.]
# [1.]
2 10.0 [0.40808206 1.        ]
# [0.40808206]
12 0.0 [-1. -1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
# [-1. -1. -1.  1.  1.  1.]
12 10.0 [-0.98404267 -0.23349587 -0.00439738  0.06330687  0.24934553  0.85167152
  1.          1.          1.          1.          1.          1.        ]
# [-0.98404267 -0.23349587 -0.00439738  0.06330687  0.24934553  0.85167152]
22 0.0 [-1. -1. -1. -1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.]
# [-1. -1. -1. -1. -1.  1.  1.  1.  1.  1.  1.]
22 10.0 [-9.99999998e-01 -9.88927789e-01 -9.36145035e-01 -1.19142115e-01
 -8.21052131e-02  2.65215399e-04  3.71042684e-03  1.34534130e-01
  6.40368363e-01  9.13174578e-01  9.99981916e-01  1.00000000e+00
  1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
  1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
  1.00000000e+00  1.00000000e+00]
# [-9.99999998e-01 -9.88927789e-01 -9.36145035e-01 -1.19142115e-01
 -8.21052131e-02  2.65215399e-04  3.7

In [7]:
## CDW 001001001... initial
## matrix size: 2L/3*2L/3
## len(matz[0,:]) = 2L/3

period = 3
J = 1.0
#ts = np.linspace(0,20.0,201)
ts = [0.0,10.0]
#Ls = np.arange(3,63,3)
Ls = [3,12,21]

for i,L in enumerate(Ls):
    rs, ks = calc_rsks(L)
    eks = calc_eks(J,ks)
    xkrs = calc_xkrs(L,ks,rs)
    system = np.array([1.0 if i<L//2 else 0.0 for i in range(L)])
    for j,t in enumerate(ts):
        #matz = calc_z(L,system,t,eks,xkrs,period=period)
        matz, fs = calc_z_2(L,system,t,eks,xkrs,period=period)
        #print(np.sum(np.abs(fs.conjugate().T-fs)))
        print(L,t,np.linalg.eigvalsh(matz))
        print("#",2.0*np.linalg.eigvalsh(fs)-1.0)

3 0.0 [1. 1.]
# [1.]
3 10.0 [-0.50495632  1.        ]
# [-0.50495632]
12 0.0 [-1. -1.  1.  1.  1.  1.  1.  1.]
# [-1. -1.  1.  1.]
12 10.0 [-0.94729631 -0.43837237  0.07408714  0.8390456   1.          1.
  1.          1.        ]
# [-0.94729631 -0.43837237  0.07408714  0.8390456 ]
21 0.0 [-1. -1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
# [-1. -1. -1.  1.  1.  1.  1.]
21 10.0 [-0.95434297 -0.81587122 -0.28227244 -0.24721954  0.16226139  0.25999137
  0.82751576  1.          1.          1.          1.          1.
  1.          1.        ]
# [-0.95434297 -0.81587122 -0.28227244 -0.24721954  0.16226139  0.25999137
  0.82751576]
